In [20]:
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from sklearn import metrics

In [2]:
df = pd.read_csv('data/train.csv').dropna()
df.drop(columns=['center_id', 'patient_id', 'image_num'], inplace=True)
df.head()

,image_id,label
0,006388_0,CE
1,008e5c_0,CE
2,00c058_0,LAA
3,01adc5_0,LAA
4,026c97_0,CE


In [3]:
CE_count = df.where(df['label'] == 'CE').count()[0]
CE_count

547

In [4]:
LAA_count = df.where(df['label'] == 'LAA').count()[0]
LAA_count

207

In [5]:
cutoff = CE_count if CE_count < LAA_count else LAA_count
cutoff

207

In [6]:
CE = df.where(df['label'] == 'CE').dropna().head(cutoff)
LAA = df.where(df['label'] == 'LAA').dropna().head(cutoff)

In [7]:
data = pd.concat([CE, LAA]).dropna()
data.head(cutoff*2)

,image_id,label
0,006388_0,CE
1,008e5c_0,CE
4,026c97_0,CE
6,029c68_0,CE
7,032f10_0,CE
...,...,...
740,fd684f_0,LAA
745,fde9ba_0,LAA
750,fe9bec_0,LAA
752,ffec5c_0,LAA


In [8]:
images = np.array([np.array(Image.open(f'jpgs/{img_id}.jpg')) for img_id in data['image_id']])
images.shape

(414, 512, 512)

In [9]:
enc_labels = np.zeros(shape=(414, 2))
for i, label in enumerate(data['label']):
    enc_labels[i][1 if label == 'CE' else 0] = 1

In [10]:
keras.utils.set_random_seed(1234)
ConvNN = keras.Sequential()
ConvNN.bias = 0.5
ConvNN.add(keras.layers.Input((512, 512)))
ConvNN.add(keras.layers.Rescaling(1.0/255))

#ConvNN.add(keras.layers.Reshape((512, 512)))

ConvNN.add(keras.layers.Convolution1D(256, (4), strides=(2), activation='relu'))
ConvNN.add(keras.layers.MaxPool1D(2))

ConvNN.add(keras.layers.Convolution1D(128, (4), strides=(2), activation='relu'))
ConvNN.add(keras.layers.MaxPool1D(2))

ConvNN.add(keras.layers.Convolution1D(64,  (4), strides=(2), activation='relu'))
ConvNN.add(keras.layers.MaxPool1D(2))

ConvNN.add(keras.layers.Flatten())
ConvNN.add(keras.layers.Dense(262144, activation='relu'))
ConvNN.add(keras.layers.Dense(512, activation='relu'))
ConvNN.add(keras.layers.Dense(256,  activation='relu'))

ConvNN.add(keras.layers.Dropout(.5))

ConvNN.add(keras.layers.Dense(2, activation='softmax'))
ConvNN.output_shape

(None, 2)

In [11]:
callbacksConv = [
    keras.callbacks.ModelCheckpoint("convNN_save_at_{epoch}.h5"),
]

In [12]:
ConvNN.compile( metrics=["accuracy"], 
optimizer = keras.optimizers.Adam(learning_rate=1e-3), 
loss="categorical_crossentropy")

In [13]:
ConvNN.fit(images, enc_labels, epochs=25, callbacks = callbacksConv)

Epoch 1/25
13/13 [==============================] - 152s 11s/step - loss: 1.5324 - accuracy: 0.4928
Epoch 2/25
13/13 [==============================] - 96s 8s/step - loss: 0.7248 - accuracy: 0.5266
Epoch 3/25
13/13 [==============================] - 85s 7s/step - loss: 0.6875 - accuracy: 0.5580
Epoch 4/25
13/13 [==============================] - 87s 7s/step - loss: 0.6716 - accuracy: 0.6014
Epoch 5/25
13/13 [==============================] - 89s 7s/step - loss: 0.6566 - accuracy: 0.5386
Epoch 6/25
13/13 [==============================] - 90s 7s/step - loss: 0.6638 - accuracy: 0.5821
Epoch 7/25
13/13 [==============================] - 88s 7s/step - loss: 0.6406 - accuracy: 0.6280
Epoch 8/25
13/13 [==============================] - 89s 7s/step - loss: 0.6248 - accuracy: 0.6570
Epoch 9/25
13/13 [==============================] - 88s 7s/step - loss: 0.6276 - accuracy: 0.6667
Epoch 10/25
13/13 [==============================] - 88s 7s/step - loss: 0.5693 - accuracy: 0.6691
Epoch 11/25
13/13

In [14]:
keras.utils.set_random_seed(1234)
ConvNN = keras.Sequential()
ConvNN.bias = 0.5
ConvNN.add(keras.layers.Input((512, 512)))
ConvNN.add(keras.layers.Rescaling(1.0/255))

ConvNN.add(keras.layers.RandomFlip(
    mode="horizontal_and_vertical", seed=1234
))




ConvNN.add(keras.layers.Convolution1D(256, (4), strides=(2), activation='relu'))
ConvNN.add(keras.layers.MaxPool1D(2))

ConvNN.add(keras.layers.Convolution1D(128, (4), strides=(2), activation='relu'))
ConvNN.add(keras.layers.MaxPool1D(2))

ConvNN.add(keras.layers.Convolution1D(64,  (4), strides=(2), activation='relu'))
ConvNN.add(keras.layers.MaxPool1D(2))

ConvNN.add(keras.layers.Flatten())
ConvNN.add(keras.layers.Dense(262144, activation='relu'))
ConvNN.add(keras.layers.Dense(512, activation='relu'))
ConvNN.add(keras.layers.Dense(256,  activation='relu'))

ConvNN.add(keras.layers.Dropout(.5))

ConvNN.add(keras.layers.Dense(2, activation='softmax'))
ConvNN.output_shape

(None, 2)

In [15]:
callbacksConv = [
    keras.callbacks.ModelCheckpoint("convNNv2_save_at_{epoch}.h5"),
]

In [16]:
ConvNN.compile( metrics=["accuracy"], 
optimizer = keras.optimizers.Adam(learning_rate=1e-3), 
loss="categorical_crossentropy")

In [17]:
ConvNN.fit(images, enc_labels, epochs=25, callbacks = callbacksConv)

13/13 [==============================] - 51s 4s/step - loss: 1.3271 - accuracy: 0.5072
Epoch 2/25
13/13 [==============================] - 48s 4s/step - loss: 0.6945 - accuracy: 0.5435
Epoch 3/25
13/13 [==============================] - 47s 4s/step - loss: 0.6797 - accuracy: 0.5700
Epoch 4/25
13/13 [==============================] - 47s 4s/step - loss: 0.6656 - accuracy: 0.5870
Epoch 5/25
13/13 [==============================] - 48s 4s/step - loss: 0.6668 - accuracy: 0.5845
Epoch 6/25
13/13 [==============================] - 48s 4s/step - loss: 0.6422 - accuracy: 0.6111
Epoch 7/25
13/13 [==============================] - 42s 3s/step - loss: 0.6439 - accuracy: 0.5990
Epoch 8/25
13/13 [==============================] - 44s 3s/step - loss: 0.5899 - accuracy: 0.6546
Epoch 9/25
13/13 [==============================] - 31s 2s/step - loss: 0.5790 - accuracy: 0.6691
Epoch 10/25
13/13 [==============================] - 31s 2s/step - loss: 0.5963 - accuracy: 0.6643
Epoch 11/25
13/13 [===========